# Section 1: Business understanding

We study used-car data to study the used-car market in the UK, and look to uderstand what determins the price of a used car. 

Question 1: Which individual features of a used car has the greatest affect on its price?

Question 2: Is it possible to predict the price of a used car based on existing data?

Question 3: Which 3 features of a used car are the best predictor of its price?

In [3]:
# import packages for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
# import glob to load .csv data files
import glob
import itertools

ModuleNotFoundError: No module named 'pandas'

# Section 2: Data Understanding

Gather, clean, and assemble data

In [2]:
## Create DataFrame by concatenating 'make' used car data files

def get_data():
    
    """
    Collects and assembles a dataframe from stored data files. This function:    
    1. Calls in all .csv files stored in the local folder and creates a temp DataFrame 'pulled'
    2. Ignores 'unclean', 'cclass', and 'focus' columns
    3. A 'make' column created from the file name is appened to 'pulled'
    4. Tax columns headers are updated to be a consistent name
    5. 'pulled' is appended to df, the collated DataFrame
    
    inputs:
    None
    
    returns:
    df: DataFrame of assembled data
    """
    
    df = pd.DataFrame()
    for file_name in glob.glob('*.csv'):

        # cclass.csv and focus.csv contain subsets of merc.csv and ford.csv, respectively. 
        # These files, along with their uncleaned versions, are not included to avoid duplication. 
        if 'unclean' not in file_name and 'cclass' not in file_name and 'focus' not in file_name:
            pulled = pd.read_csv(file_name, low_memory=False)
            
             # create 'make' column by dropping '.csv' part of file_name
            pulled['make'] = file_name[:-4]

            # tax is given as 'tax' in some files and 'tax(£)' in others. 
            # We rename all columns to 'tax' for consistency. 
            try:
                pulled.rename(columns = {'tax(£)':'tax'}, inplace = True)
            except:
                continue
            df = pd.concat([df,pulled],axis=0)
    
    return df

df = get_data()
df.shape
df.head()

NameError: name 'pd' is not defined

#  Section 3: Data Preparation

Cleaning data by recasting 'year' feature to new 'age' feature, to aid interpretation of results. 

In [ ]:
# Recast year column as 'age' to make interpretation easier, and drop 'year' column. 

df['age'] = 2020 - df['year']
df.drop(['year'], axis=1, inplace=True)


Create dummy columns for catagorical features (car model, transmission type, fuel type). 

In [ ]:
# Extract catagorical columns to be recast a dummy columns

cat_df = df.select_dtypes(include=['object']).copy()
cat_cols = cat_df.columns

In [ ]:
# Create numerical-only dataframe by adding dummy columns for categorical data. 

def create_dummy_df(df, cat_cols):
    
    """
    Converts catagorical columns to dummy columns. This function:
    1. Loops over each column name in cat_cols
    2. For each column, dummy columns are created
    3. New dummy columns are appended
    4. Original catagorical column is dropped
    
    Inputs:
    df: used-car data set
    cat_cols: list of catagorical columns headers
    
    Returns:
    df: DataFrame with dummy columns
    """

    for col in cat_cols:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True)], axis=1)
        except:
            continue
    return df.astype(np.float)

dum_df = create_dummy_df(df, cat_cols)
dum_df.shape[0]

Rows:Columns still comfortably accommodates 10:1 requirement, so we are able to continue using all catagorical dummy columns. 

# Section 4: Modelling

We train and test a linear regression model to predict used-car price. 

In [ ]:
# Modelling using all features is computationally time-consuming 
# We facilitate adding and removing features to speed up model training

# These lists contain groups of features to aid investigation
# Lists of makes, models, transmission, and fuel created by string-matching columns in dum_df
model_list = [col for col in dum_df.columns if 'model' in col]
make_list  = [col for col in dum_df.columns if 'make' in col]
trans_list = [col for col in dum_df.columns if 'transmission' in col]
fuel_list  = [col for col in dum_df.columns if 'fuelType' in col]
mil_list = ['mileage']
tax_list = ['tax']
mpg_list = ['mpg']
eng_list = ['engineSize']
age_list = ['age']


# subset features to train linear regression model
# users should remove from this list features which are not to be used for training

feat_list = [mil_list, eng_list, age_list, tax_list, mpg_list, make_list, model_list, trans_list, fuel_list]


In [ ]:
# This function returns a concise string for the user-chosen features used in the model training. 

def get_feat_output(feature_list):
    
    """
    This function creates a string describing the features included to aid investigation:
    1. feature_list is checked for appropriate strings
    2. A string 'output_list' is built and returned
    
    Inputs:
    feature_list: list of features used for analysis
    
    Returns:
    output_list: description string
    """
    
    output_list = ''
    if 'mileage' in feature_list:
        output_list += 'miles_'
    if 'tax' in feature_list:
        output_list += 'tax_'
    if 'mpg' in feature_list:
        output_list += 'mpg_'
    if 'engineSize' in feature_list:
        output_list += 'eng_'
    if 'age' in feature_list:
        output_list += 'age_'
    if 'make_bmw' in feature_list:
        output_list += 'make_'
    if 'model_ 2 Series' in feature_list:
        output_list += 'model_'
    if 'transmission_Manual' in feature_list:
        output_list += 'trans_'
    if 'fuelType_Electric' in feature_list:
        output_list += 'fuel_'
        
    return output_list

In [ ]:
# Loop over combinations of above lists - 

# User-required number of features. 
# E.g. when num_feats=3, the model will train on all combinations of 3 features from feature_train_list
num_feats = 2
result_df = pd.DataFrame(columns = ['features','r2_score']) # results of each training will be stored here

# iter_tools.combinations provides all combinations of num_feats features
for feat_set in itertools.combinations(feat_list, num_feats):

    # Convert feature set into list and get string for features used in training
    feature_list = list(feat_set)
    # flatten feature_list
    feature_list = [item for sublist in feature_list for item in sublist]
    output_str = get_feat_output(feature_list)

    # Split into explanatory and response variables
    X = dum_df[feature_list]
    y = dum_df['price']

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42)

    # Instantiate and fit linear regression model
    lm_model = LinearRegression(normalize=True)
    lm_model.fit(X_train, y_train) #Fit

    # Predict and score the model
    y_test_preds = lm_model.predict(X_test)
    result_df.loc[len(result_df)] = [output_str[:-1],r2_score(y_test, y_test_preds)]

    #result_list.append([output_str[:-1],r2_score(y_test, y_test_preds)])
    print("For features [{}]: the r-squared score was {} on {} values.".format(output_str[:-1], r2_score(y_test, y_test_preds), len(y_test)))


# Section 5: Evaluate

Question 1: What features of a used car has the greatest affect on its price?

We create a heatmap for a correlation matrix, which will show which features correlate most (either positively or negatively) with price. 

In [ ]:
# Create heatmap for correlation matrix to study correlation between numerical columns

sns.heatmap(df.corr(), annot=True, fmt='.2f')
plt.show()

Evaluation for Question 1:

The strongest positive correlation for price is between price and engine size, so a larger engine size is the best single indicator of a higher price. 

The strongest negative correlation for price is between price and mileage, and price and age. So older cars, or cars with a higher mileage indicate a lower price when other features are kept constant. 

Question 2: Is it possible to predict the price of a used car based on existing data?

Evaluation for Question 2:

Setting num_feats = len(feat_list) and running train/test loop, we get:
For features [miles_tax_mpg_eng_age_make_model_trans_fuel]: the r-squared score was 0.8693213456408345 on 9919 values. 

This confirms that the linear-regression model is successfully trained on the training data, and has a good r2 score. We can use exiting data to predict the price of a user car based on all features available. 


Question 3: Which 3 features of a used car are the best predictor of its price?

In [ ]:
# setting num_feats = 3
result_df = result_df.sort_values(by=['r2_score'], ascending=False)

# take top 5 and bottom 5 feature combinations by r2_score
result_top5 = result_df[:5]
result_bottom5 = result_df[-5:]
top_bottom_df = pd.concat([result_top5, result_bottom5], axis=0)
top_bottom_df.plot.bar(x='features', y='r2_score', rot=0)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

Evaluation for Question 3:

With an r2 score of 0.826, mileage, engineSize, and model is the best combination for predicting used-car price. 
With an r2 score of 0.241, mileage, tax, and mpg is the worst combination for predicting price. 